In [ ]:
import pickle
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Initialize WebDriver
chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open Instagram login page
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)

# Handle consent buttons
consent_button_xpaths = [
    "//button[contains(text(),'Allow all cookies')]",
    "//button[contains(text(),'Accept')]",
    "//button[contains(text(),'Consent')]",
    "//button[contains(text(),'Agree')]"
]

for xpath in consent_button_xpaths:
    try:
        consent_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        consent_button.click()
        print(f"Clicked consent button: {xpath}")
        time.sleep(2)
        break
    except:
        continue

# Login
username = ""
password = ""

username_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
password_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "password"))
)

username_input.send_keys(username)
password_input.send_keys(password)

login_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
)
login_button.click()

# Wait for login to complete
time.sleep(5)

# Save cookies
cookies = driver.get_cookies()
with open("cookies.pkl", "wb") as file:
    pickle.dump(cookies, file)
print("Cookies saved!")

# Quit WebDriver
driver.quit()


Clicked consent button: //button[contains(text(),'Allow all cookies')]
Cookies saved!


In [ ]:
import pickle
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Instagram pages to scrape
instagram_pages = [
    "https://www.instagram.com/valorgi/",
    "https://www.instagram.com/hotfreestyle/"
]

# Initialize WebDriver
chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open Instagram to apply cookies
driver.get("https://www.instagram.com/")
time.sleep(5)

# Load cookies
try:
    with open("cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)
    print("Cookies loaded successfully!")
except FileNotFoundError:
    print("No saved cookies found! Please log in first.")

# Refresh the page to apply cookies
driver.refresh()
time.sleep(5)

# Function to extract posts
def extract_posts(soup):
    posts = []
    post_containers = soup.find_all("div", {"class": "x1lliihq x1n2onr6 xh8yej3 x4gyw5p x11i5rnm x1ntc13c x9i3mqj x2pgyrj"})
    
    for container in post_containers[:5]:  # Limit to first 5 posts
        post_data = {}
        
        # Extract post URL
        try:
            post_url = "https://www.instagram.com" + container.find("a")["href"]
            post_data["post_url"] = post_url
        except:
            post_data["post_url"] = "No URL"
        
        # Extract caption from the <img> tag inside the container
        try:
            img_tag = container.find("img", {"class": "x5yr21d xu96u03 x10l6tqk x13vifvy x87ps6o xh8yej3"})
            caption = img_tag["alt"]
            post_data["caption"] = caption
        except:
            post_data["caption"] = "No caption"
        
        posts.append(post_data)
    
    return posts

# Scrape content from each Instagram page
output = {}
for page in instagram_pages:
    print(f"Scraping: {page}")
    driver.get(page)
    time.sleep(5)

    # Wait for profile page to load
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
    except:
        print(f"Failed to load {page}")
        continue

    # Get page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract posts
    posts = extract_posts(soup)
    output[page] = posts

# Close WebDriver
driver.quit()

# Print JSON output with ensure_ascii=False
print(json.dumps(output, indent=4, ensure_ascii=False))

Cookies loaded successfully!
Scraping: https://www.instagram.com/valorgi/
Scraping: https://www.instagram.com/hotfreestyle/
{
    "https://www.instagram.com/valorgi/": [
        {
            "post_url": "https://www.instagram.com/valorgi/reel/DGq3X4AOboe/",
            "caption": "This person transformed a desert into a city of the future in just 50 years.\n⠀\nWe help people improve their lives through business and educational content.\n⠀\nJoin us on our journey 🔸 @Valorgi\n⠀\n⠀\nDM for credit or removal request (no copyright intended for footage video and music) © All rights and credits reserved to the respective owner(s)"
        },
        {
            "post_url": "https://www.instagram.com/valorgi/p/DGqkmTYO4Gh/",
            "caption": "Photo by Valorgi™ | Business | Wisdom on March 01, 2025. May be a meme of 3 people, people standing, people kissing, poster and text that says '滿 MAos VALORGI WHEN YOU TWO GET INTO A FIGHT, REMEMBER, IT'S YOU AND HER VS THE PROBLEM. NOT YOU VS HE

In [3]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
from io import BytesIO

# Set up Chrome WebDriver
def get_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Run in background
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Function to perform reverse image search
def reverse_image_search(image_path, source="google"):
    driver = get_driver()
    
    if source == "google":
        driver.get("https://www.google.com/imghp?hl=en")
        time.sleep(2)

        # Click on camera icon (Google)
        camera_button = driver.find_element(By.CLASS_NAME, "nDcEnd")
        camera_button.click()
        time.sleep(2)

        # Upload image
        upload_input = driver.find_element(By.NAME, "encoded_image")
        upload_input.send_keys(os.path.abspath(image_path))
        time.sleep(5)  # Wait for search results to load

    elif source == "bing":
        driver.get("https://www.bing.com/visualsearch")
        time.sleep(2)

        # Upload image (Bing)
        upload_button = driver.find_element(By.ID, "sb_img_input")
        upload_button.send_keys(os.path.abspath(image_path))
        time.sleep(5)

    else:
        print("❌ Invalid source! Use 'google' or 'bing'.")
        return []

    # Scrape image URLs from search results
    img_elements = driver.find_elements(By.TAG_NAME, "img")
    img_urls = [img.get_attribute("src") for img in img_elements if img.get_attribute("src")]

    driver.quit()
    
    return img_urls[:5]  # Return top 5 image URLs

# Function to download images
def download_images(img_urls, folder="search_results"):
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i, url in enumerate(img_urls):
        try:
            response = requests.get(url)
            image = Image.open(BytesIO(response.content))
            image.save(f"{folder}/result_{i+1}.jpg")
            print(f"✅ Saved: result_{i+1}.jpg")
        except Exception as e:
            print(f"❌ Failed to save image {i+1}: {e}")

# Run the reverse image search
image_path = "https://m.media-amazon.com/images/M/MV5BNzRiOTRiNmEtMWQxOC00NjY2LTk0NGYtYjA1OGFjOGY4OTdkXkEyXkFqcGc@._V1_FMjpg_UX1000_.jpg"  # Change this to your image file
search_source = "bing"  # Change to "bing" if needed

image_urls = reverse_image_search(image_path, source=search_source)
download_images(image_urls)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="sb_img_input"]"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x01120B43+25139]
	(No symbol) [0x010B13F4]
	(No symbol) [0x00F904E3]
	(No symbol) [0x00FD83D7]
	(No symbol) [0x00FD872B]
	(No symbol) [0x01021002]
	(No symbol) [0x00FFD014]
	(No symbol) [0x0101E778]
	(No symbol) [0x00FFCDC6]
	(No symbol) [0x00FCBDE9]
	(No symbol) [0x00FCD124]
	GetHandleVerifier [0x01424373+3185251]
	GetHandleVerifier [0x0144291A+3309578]
	GetHandleVerifier [0x0143CF42+3286578]
	GetHandleVerifier [0x011B7AE0+643536]
	(No symbol) [0x010BA20D]
	(No symbol) [0x010B70B8]
	(No symbol) [0x010B7257]
	(No symbol) [0x010A9E00]
	BaseThreadInitThunk [0x7599FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776282AE+286]
	RtlGetAppContain

No cookies found or error loading cookies


In [11]:
cl = Client()
cl.login(config.scoopy_doom, config.ekeprecious)

AttributeError: module 'config' has no attribute 'scoopy_doom'

In [1]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui

# Setup Chrome options with mobile emulation
mobile_emulation = {"deviceName": "Nexus 5"}
options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open Instagram login page
driver.get("https://www.instagram.com/accounts/login/")

# Wait for the page to load
time.sleep(3)

# Load cookies from cookies.pkl
cookies = pickle.load(open("cookies.pkl", "rb"))

# Add cookies to the browser session
for cookie in cookies:
    # Cookies need to have the 'domain' set to Instagram's domain
    driver.add_cookie(cookie)

# Refresh the page to authenticate using the loaded cookies
driver.refresh()

# Wait for the page to load after cookies are added
time.sleep(5)

# Handle consent buttons
consent_button_xpaths = [
    "//button[contains(text(),'Allow all cookies')]",
    "//button[contains(text(),'Accept')]",
    "//button[contains(text(),'Consent')]",
    "//button[contains(text(),'Agree')]"
]

for xpath in consent_button_xpaths:
    try:
        consent_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        consent_button.click()
        print(f"Clicked consent button: {xpath}")
        time.sleep(2)
        break
    except:
        continue

# Click on the "+" button to upload a photo
upload_button = driver.find_element(By.XPATH, "//div[@aria-label='New Post']")
upload_button.click()

# Wait for the file input to open
time.sleep(2)

# Use pyautogui to open the file dialog and select an image
pyautogui.write(r"C:\Users\valen\Desktop\SocialHub\image3.png")
pyautogui.press('enter')

# Wait for the upload to complete
time.sleep(3)

# Click on the 'Next' button
next_button = driver.find_element(By.XPATH, "//button[text()='Next']")
next_button.click()

# Wait for the description input field
time.sleep(2)

# Add caption, tags, or description
caption_field = driver.find_element(By.XPATH, "//textarea[@aria-label='Write a caption…']")
caption_field.send_keys("Your caption here #hashtag")

# Click on 'Share' to post
share_button = driver.find_element(By.XPATH, "//button[text()='Share']")
share_button.click()

# Wait a few seconds before closing the browser
time.sleep(5)

# Close the browser
input("Press Enter to close the browser...")
driver.close()


ModuleNotFoundError: No module named 'pyautogui'

In [2]:
pip install pyautogui


^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup Chrome options with mobile emulation
mobile_emulation = {"deviceName": "Nexus 5"}
options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open Instagram login page
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)  # Allow time for the page to load

# Load cookies from cookies.pkl
cookies = pickle.load(open("cookies.pkl", "rb"))

# Add cookies to the browser session
for cookie in cookies:
    driver.add_cookie(cookie)

# Refresh the page to authenticate using the loaded cookies

time.sleep(5)  # Wait for the page to load

# Click on "Home" button to ensure UI is fully loaded
try:
    driver.refresh()
    home_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'x9f619 xxk0z11 xii2z7h x11xpdln x19c4wfv xvy4d1p')]"))
    )
    home_button.click()
    print("Clicked Home button.")
    time.sleep(3)
except Exception as e:
    print("Home button not found:", e)

# Click on "Post" button
try:
    post_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Post')]"))
    )
    post_button.click()
    print("Clicked Post button.")
    time.sleep(3)
except Exception as e:
    print("Post button not found:", e)

# Manual file selection step
input("Select your image manually and press Enter to continue...")

# Wait for "Next" button to appear and click it
try:
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='_ab18 _ab1b']/button[@class='_ab5p']"))
    )
    next_button.click()
    print("Clicked Next button.")
    time.sleep(3)
except Exception as e:
    print("Next button not found:", e)

# Enter caption ("Hello")
try:
    caption_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='_abrw']/textarea"))
    )
    caption_field.send_keys("Hello")
    print("Entered caption.")
    time.sleep(3)
except Exception as e:
    print("Caption field not found:", e)

# Click "Share" button
try:
    share_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='_ab18 _ab1b']/button[@class='_ab5p']"))
    )
    share_button.click()
    print("Clicked Share button.")
    time.sleep(5)
except Exception as e:
    print("Share button not found:", e)

# Close the browser
print("Post uploaded successfully!")


Clicked Home button.
Clicked Post button.
Clicked Next button.
Entered caption.
Clicked Share button.
Post uploaded successfully!


In [3]:
import requests

# Replace with your actual API key
API_KEY = "e53a1008dfe74b1e9244b63e99244d96"

# Path to your image file
input_image_path = r"C:\Users\valen\Desktop\SocialHub\image3.png"
output_image_path = "enhanced_image.png"

# API endpoint
url = "https://www.cutout.pro/api/v1/photoEnhance"

# Prepare the request
headers = {
    "APIKEY": API_KEY
}

# Open the image file
files = {
    "file": open(input_image_path, "rb")
}

# Make the API request
response = requests.post(
    url,
    headers=headers,
    files=files
)

# Check if the request was successful
if response.status_code == 200:
    # Save the enhanced image
    with open(output_image_path, "wb") as out_file:
        out_file.write(response.content)
    print(f"Enhanced image saved to {output_image_path}")
else:
    # Print error message
    print(f"Error: {response.status_code}")
    print(response.text)

Enhanced image saved to enhanced_image.png
